![astropy logo](http://astropy.readthedocs.org/en/stable/_images/astropy_banner.svg)

<br /><br />


# [`astroquery`](https://astroquery.readthedocs.io) <a id='astroquery'></a>



`astroquery` allows you to query astronomical databases with ease. 



## Table of contents
* [astroquery](#astroquery)
  * [SIMBAD queries](#simbad)
  * [Vizier queries](#vizier)

<br /><br />

### SIMBAD: [`astroquery.simbad`](http://astroquery.readthedocs.io/en/latest/simbad/simbad.html)<a id='simbad'></a>


Suppose you're about to observe the exoplanet host star HD 189733, and you'd like to know its: 

* position
* $B$ and $V$ colors
* spectral type

All of this information is easily accessible via the [SIMBAD Astronomical Database](http://simbad.u-strasbg.fr).

Let's query for the SIMBAD for this object (a star):

In [ ]:
from astroquery.simbad import Simbad

identifier = 'HD 189773'

result = Simbad.query_object(identifier)
result

You can see that some basic information is returned in an [`Table`](http://docs.astropy.org/en/stable/table/) object. We can create an [`SkyCoord`](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html#astropy.coordinates.SkyCoord) object to describe its position by accessing the `ra` and `dec` columns:

In [ ]:
from astropy.coordinates import SkyCoord, Angle
import astropy.units as u

# Use astropy.coordinates.Angle class to parse the position strings:
ra = Angle(result['RA'][0], u.hourangle)
dec = Angle(result['DEC'][0], u.degree)

# Initialize a astropy.coordinates.SkyCoord object for HD 189733
position = SkyCoord(ra=ra, dec=dec)

print(position)

The default SIMBAD query table includes position and proper motion, but doesn't include the colors and spectral type. You can add custom query parameters if you know their field names, [which are defined here](http://simbad.u-strasbg.fr/simbad/sim-help?Page=sim-fscript#VotableFields):

In [ ]:
from astroquery.simbad import Simbad

# We're going to create a custom instance of the SIMBAD class:
s = Simbad()

# We're going to add fields to the VOTable for spectral type:
s.add_votable_fields('sptype')

# ...and for the B and V magnitudes: 
s.add_votable_fields('flux(B)')
s.add_votable_fields('flux(V)')

# Now let's query for the object on our new SIMBAD instance:
result = s.query_object(identifier)
result

The properties we requested are listed in the columns `SP_TYPE`, `FLUX_B` and `FLUX_V`.

In [ ]:
sptype = result['SP_TYPE'][0].decode()
b = result['FLUX_B'][0]
v = result['FLUX_V'][0]

print("Spectral type: {0}".format(sptype))
print("B = {0}".format(b))
print("V = {0}".format(v))

### Vizier: [`astroquery.vizier`](http://astroquery.readthedocs.io/en/latest/vizier/vizier.html)<a id='vizier'></a>

Let's query Vizier for the famous list of standard stars from [Landolt (1992)](http://adsabs.harvard.edu/abs/1992AJ....104..340L). The [`Table`](http://astropy.readthedocs.io/en/latest/table/) that is returned to you will have the same information as [this Vizier query page](http://vizier.u-strasbg.fr/viz-bin/VizieR-3?-source=II/183A/table2).

In [ ]:
from astroquery.vizier import Vizier

# Generic search using author and year format:
table_list = Vizier.get_catalogs('Landolt 1992')

table_list

What gets returned is a `TableList` object, containing all of the tables that matched our (vague) query. You can see that astroquery will return the first 50 rows of each table by default. 

You could also search for a specific table by its catalog identifier, which is "II/183A/table2" for this table. We could have searched via catalog ID like this: 

In [ ]:
identifier = 'II/183A/table2'

Vizier.get_catalogs(catalog=identifier)

There's only one table this time, because this identifier is unique. Now let's get all rows of the table, instead of the first 50: 

In [ ]:
# Set the row limit to a very large number:
Vizier.ROW_LIMIT = 1e10

# Get the first table:
result = Vizier.get_catalogs(catalog=identifier)[0]

result

This returns all 526 standard stars in the table. Let's see the distribution of colors, and the distribution on the sky:

In [ ]:
import numpy as np

fig = plt.figure(figsize=(12, 5))
ax0 = fig.add_subplot(121)
ax1 = fig.add_subplot(122, projection='hammer')

# Plot histogram of B-V colors
ax0.hist(result['B-V'])
ax0.set_xlabel('$B-V$')
ax0.set_aspect(1e-2)

# Plot sky positions
ra = Angle(result['RAJ2000'], u.hourangle).wrap_at(np.pi*u.rad)
dec = Angle(result['DEJ2000'], u.degree)
ax1.scatter(ra.radian, dec.radian, marker='.')
ax1.grid()
plt.show()